# Task 3: Baseline Model Comparison (Decision Tree)

In this notebook, you will train a Decision Tree model to predict the `prior_hiring_decision` target variable. You should compare its performance (Accuracy and Fairness) to the Logistic Regression baseline.

In [95]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV


### **Equalized Odds**


In [90]:
def equalized_odds(data, pred_col='prediction', true_col='prior_hiring_decision'):
    """
    Compute equalized odds for binary race (white=1 vs non-white=anything else).
    TPR = P(pred=1 | true=1), FPR = P(pred=1 | true=0)
    """
    data = data.copy()
    data['_race_binary'] = (data['Race'] == 1).astype(int)
    
    tpr_rates = []
    fpr_rates = []
    
    for race in [0, 1]:  # 0=non-white, 1=white
        group_data = data[data['_race_binary'] == race]
        
        # Actual positives and negatives
        actual_positives = group_data[group_data[true_col] == 1]
        actual_negatives = group_data[group_data[true_col] == 0]
        
        # TPR: of actual positives, how many predicted positive?
        if len(actual_positives) > 0:
            tpr = actual_positives[pred_col].mean()
            tpr_rates.append(tpr)
        
        # FPR: of actual negatives, how many predicted positive?
        if len(actual_negatives) > 0:
            fpr = actual_negatives[pred_col].mean()
            fpr_rates.append(fpr)
    
    max_tpr, min_tpr = max(tpr_rates), min(tpr_rates)
    max_fpr, min_fpr = max(fpr_rates), min(fpr_rates)
    
    tpr_ratio = float('inf') if min_tpr == 0 else max_tpr / min_tpr
    fpr_ratio = float('inf') if min_fpr == 0 else max_fpr / min_fpr
    
    return max(tpr_ratio, fpr_ratio)

## 1. Preprocessing function

In [91]:

# Load Data
train_df = pd.read_csv('../data/train.csv')
val_df = pd.read_csv('../data/val.csv')
test_df = pd.read_csv('../data/test.csv')

# Configuration: Toggle between weighted and unweighted training
USE_WEIGHTED = False

columns_to_drop = ["Hours_Per_Week", "Marital_Status", "Relationship"]
protected_characteristics = ["Sex", "Race", "Age", "Place_Of_Birth"]
target = 'prior_hiring_decision'

# Drop columns_to_drop (but keep protected characteristics for now - needed for weights)
for df in [train_df, val_df, test_df]:
    df.drop(columns=columns_to_drop, inplace=True)

print(f"Training mode: {'WEIGHTED' if USE_WEIGHTED else 'UNWEIGHTED'}")
print("Training Shape:", train_df.shape)

Training mode: UNWEIGHTED
Training Shape: (30000, 10)


In [92]:
def compute_group_weights(df, label_col):
    """
    Compute rebalancing weights for each (race_binary, label) combination.
    Returns a dictionary mapping (race_binary, label) -> weight.
    
    Weight formula: w = P(race) * P(label) / P(race, label)
    This makes race and label independent in the weighted distribution.
    """
    total = len(df)
    labels = df[label_col].unique()
    
    # Binary race: 1 = white, 0 = non-white
    race_binary = (df['Race'] == 1).astype(int)
    
    weights = {}
    for race in [0, 1]:
        race_count = (race_binary == race).sum()
        for label in labels:
            label_count = (df[label_col] == label).sum()
            intersection_count = ((race_binary == race) & (df[label_col] == label)).sum()
            
            if intersection_count > 0:
                # w = P(race) * P(label) / P(race, label)
                weight = (race_count * label_count) / (total * intersection_count)
            else:
                weight = 1.0
            
            weights[(race, int(label))] = float(weight)
    
    return weights


def get_sample_weights(df, group_weights, label_col):
    """
    Get per-sample weights array for use in loss function.
    """
    race_binary = (df['Race'] == 1).astype(int)
    sample_weights = [
        group_weights.get((race, int(label)), 1.0)
        for race, label in zip(race_binary, df[label_col])
    ]
    return np.array(sample_weights)


# Compute group weights from training data
group_weights = compute_group_weights(train_df, target)

print("Group Weights (applied to loss function):")
print("  Formula: w = P(race) * P(label) / P(race, label)")
print()
for (race, label), weight in sorted(group_weights.items()):
    race_name = "white" if race == 1 else "non-white"
    label_name = "hired" if label == 1 else "not hired"
    print(f"  {race_name}, {label_name}: {weight:.4f}")

Group Weights (applied to loss function):
  Formula: w = P(race) * P(label) / P(race, label)

  non-white, not hired: 0.8388
  non-white, hired: 1.2625
  white, not hired: 1.1321
  white, hired: 0.8879


## 2. Preprocessing Fairness

In [ ]:
# 1. Prepare sample weights
if USE_WEIGHTED:
    sample_weights = get_sample_weights(train_df, group_weights, target)
else:
    sample_weights = None

# 2. Define features and split (using raw DataFrames)
cols_to_exclude = protected_characteristics + [target]
features = [col for col in train_df.columns if col not in cols_to_exclude]

X_train, y_train = train_df[features], train_df[target]
X_val, y_val = val_df[features], val_df[target]
x_test, y_test = test_df[features], test_df[target]

# 3. Build the Pipeline
# We keep the raw data until it enters the pipeline
categorical_cols = [col for col in ['Workclass', 'Education', 'Occupation'] if col in features]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ],
    remainder='passthrough'
)

full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# 4. Hyperparameter grid (Note the 'classifier__' prefix)
param_grid = {
    'classifier__max_depth': [10, 15, None],
    'classifier__min_samples_split': [5, 10],
    'classifier__min_samples_leaf': [2, 4]
}

print(f"\nTraining with {'SAMPLE WEIGHTS' if USE_WEIGHTED else 'NO WEIGHTS'} via GridSearchCV...\n")

# 5. Execute Grid Search
# CV=5 provides a more robust estimate of performance than a single validation split
random_search = RandomizedSearchCV(
    full_pipeline,
    param_grid,
    n_iter=1,      
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    random_state=42 
)

# Fit using raw X_train; weights are passed directly to the classifier step
random_search.fit(
    X_train, 
    y_train, 
    classifier__sample_weight=sample_weights
)

# 6. Final Evaluation
best_pipe = random_search.best_estimator_
val_accuracy = best_pipe.score(X_train, y_train)

print(f"Best parameters: {random_search.best_params_}")
print(f"Internal CV Score (Mean): {random_search.best_score_:.4f}")
print(f"Final Eval Accuracy (on X_train): {val_accuracy:.4f}")


Training with NO WEIGHTS via GridSearchCV...

Best parameters: {'classifier__min_samples_split': 5, 'classifier__min_samples_leaf': 4, 'classifier__max_depth': None}
Internal CV Score (Mean): 0.6705
Final Eval Accuracy (on X_val): 0.6718


## 3. Evaluation Metrics

In [ ]:
y_pred = best_pipe.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Validation Accuracy: 0.6718
Confusion Matrix:
[[3764 1435]
 [1847 2954]]


## 4. Fairness Metric Implementation

In [99]:
val_with_preds = val_df.copy()
val_with_preds['prediction'] = best_pipe.predict(X_val)

# Compute equalized odds for binary race (white vs non-white)
eo_ratio = equalized_odds(val_with_preds, pred_col='prediction', true_col=target)

# Get detailed TPR/FPR by group
val_with_preds['_race_binary'] = (val_with_preds['Race'] == 1).astype(int)
results = {}
for race in [0, 1]:
    group_data = val_with_preds[val_with_preds['_race_binary'] == race]
    
    # Actual positives and negatives (based on true label)
    actual_positives = group_data[group_data[target] == 1]
    actual_negatives = group_data[group_data[target] == 0]
    
    race_name = "white" if race == 1 else "non-white"
    results[race_name] = {
        # TPR: of actual positives, fraction predicted positive
        'tpr': float(actual_positives['prediction'].mean()) if len(actual_positives) > 0 else None,
        # FPR: of actual negatives, fraction predicted positive
        'fpr': float(actual_negatives['prediction'].mean()) if len(actual_negatives) > 0 else None,
        'count': len(group_data)
    }

print("Fairness Metrics (Decision Tree) - Binary Race:")
print(f"\nEqualized Odds Ratio: {eo_ratio:.4f}")
print("\nBy Group:")
for group, metrics in results.items():
    print(f"  {group}: TPR={metrics['tpr']:.4f}, FPR={metrics['fpr']:.4f}, Count={metrics['count']}")

Fairness Metrics (Decision Tree) - Binary Race:

Equalized Odds Ratio: 1.6178

By Group:
  non-white: TPR=0.5403, FPR=0.2049, Count=3756
  white: TPR=0.6486, FPR=0.3315, Count=6244


## 5. ROC Curve Analysis

In [1]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

y_proba = best_pipe.predict_proba(x_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
auc_score = roc_auc_score(y_test, y_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Decision Tree (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Decision Tree')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

print(f"AUC Score: {auc_score:.4f}")

NameError: name 'best_pipe' is not defined

## 6. Adjusted Equalized Odds

In [101]:
def adjusted_equalized_odds(data, y_true_col, y_proba, thresholds):
    """
    Calculate equalized odds using group-specific classification thresholds.
    Uses binary race: white (Race==1) vs non-white (Race!=1).
    
    Args:
        data: DataFrame with Race column and true labels
        y_true_col: Name of true label column
        y_proba: Array of predicted probabilities
        thresholds: Dict mapping race_binary (0 or 1) to classification threshold
                    e.g., {0: 0.4, 1: 0.6} means non-white uses 0.4, white uses 0.6
    
    Returns:
        Dict with TPR/FPR per group and equalized odds ratio
    """
    data = data.copy()
    data['y_proba'] = y_proba
    data['_race_binary'] = (data['Race'] == 1).astype(int)
    
    # Apply group-specific thresholds
    data['adjusted_pred'] = data.apply(
        lambda row: 1 if row['y_proba'] >= thresholds.get(row['_race_binary'], 0.5) else 0,
        axis=1
    )
    
    tpr_rates = []
    fpr_rates = []
    results = {}
    
    for race in [0, 1]:  # 0=non-white, 1=white
        group_data = data[data['_race_binary'] == race]
        actual_positives = group_data[group_data[y_true_col] == 1]
        actual_negatives = group_data[group_data[y_true_col] == 0]
        
        tpr = actual_positives['adjusted_pred'].mean() if len(actual_positives) > 0 else 0
        fpr = actual_negatives['adjusted_pred'].mean() if len(actual_negatives) > 0 else 0
        
        tpr_rates.append(tpr)
        fpr_rates.append(fpr)
        
        race_name = "white" if race == 1 else "non-white"
        results[race_name] = {
            'tpr': float(tpr),
            'fpr': float(fpr),
            'threshold': float(thresholds.get(race, 0.5)),
            'count': int(len(group_data))
        }
    
    max_tpr, min_tpr = max(tpr_rates), min(tpr_rates)
    max_fpr, min_fpr = max(fpr_rates), min(fpr_rates)
    
    tpr_ratio = max_tpr / min_tpr if min_tpr > 0 else float('inf')
    fpr_ratio = max_fpr / min_fpr if min_fpr > 0 else float('inf')
    
    results['tpr_ratio'] = float(tpr_ratio)
    results['fpr_ratio'] = float(fpr_ratio)
    results['equalized_odds_ratio'] = float(max(tpr_ratio, fpr_ratio))
    
    return results

In [102]:
# Get predicted probabilities
y_proba = best_pipe.predict_proba(X_val)[:, 1]
# Define group-specific thresholds to try to equalize odds
# Key: 0 = non-white, 1 = white
# Adjust these thresholds to balance TPR/FPR across groups
thresholds = {
    0: 0.31,  # lower threshold for non-white (increases their TPR)
    1: 0.5   # standard threshold for white
}

# Compute adjusted equalized odds
results = adjusted_equalized_odds(val_df, target, y_proba, thresholds)

print("Adjusted Equalized Odds (with group-specific thresholds):")
print(f"\nThresholds: non-white={thresholds[0]}, white={thresholds[1]}")
print(f"\nEqualized Odds Ratio: {results['equalized_odds_ratio']:.4f}")
print(f"TPR Ratio: {results['tpr_ratio']:.4f}")
print(f"FPR Ratio: {results['fpr_ratio']:.4f}")
print("\nBy Group:")
for group in ['non-white', 'white']:
    m = results[group]
    print(f"  {group}: TPR={m['tpr']:.4f}, FPR={m['fpr']:.4f}, threshold={m['threshold']}, count={m['count']}")

Adjusted Equalized Odds (with group-specific thresholds):

Thresholds: non-white=0.31, white=0.5

Equalized Odds Ratio: 1.1697
TPR Ratio: 1.0474
FPR Ratio: 1.1697

By Group:
  non-white: TPR=0.6919, FPR=0.3519, threshold=0.31, count=3756
  white: TPR=0.7247, FPR=0.4116, threshold=0.5, count=6244
